<a href="https://colab.research.google.com/github/Jachanya/Unsupervised-deep-learning/blob/main/transformerLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import collections
from typing import Sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys,random,math
from collections import Counter
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")

In [28]:
SOS_token = 0
EOS_token = 1
PAD_token = 2
UNK_token = 3

class Lang:
    def __init__(self, name, filename):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "<SOS>", 1: "<EOS>", 2: "<PAD>", 3: "<UNK>"}
        self.n_words = 4  # Count SOS and EOS
        self.make_vocab(filename)

    def make_vocab(self,filename):
      with open(filename, encoding = 'utf-8') as file:
        new_file = file.read()
        for i in range(len(new_file)//1000000):
          self.addSentence(new_file[i*1000000:i*1000000 + 1000000])
        #for sentence in file:
        #  self.addSentence(sentence)
      print('done')

    def addSentence(self, sentence):
        doc = nlp(sentence)
        for token in doc:
          self.addWord(token.text.lower())

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def getWord(self, word):
      if word not in self.word2index:
        return UNK_token
      
      else:
        return self.word2index[word]
    
    def getIndex(self, index):
      return self.index2word[index]

    def getSentence(self, sentence):
      doc = nlp(sentence)
      new_sentence = []
      for token in doc:
        new_sentence.append(self.getWord(token.text.lower()))
      return new_sentence 
    
    def getListIndex(self, index):
      sentence = []
      for idx in index:
        sentence.append(self.getIndex(idx))
      return sentence

language = Lang('English', 'data/shakespeare.txt')

done


In [32]:
vocab_size = len(language.index2word)

In [33]:
vocab_size

26215

In [34]:
'''
def generate_seq(filename, language):
  with open(filename, encoding ='utf-8') as file:
    sentences = []
    for sentence in file.read().split("\n"):
      #print(word)
      if len(sentence.strip()) > 4:
        sentences.append(torch.tensor(language.getSentence(sentence.strip())))
  return sentences
'''
  

'\ndef generate_seq(filename, language):\n  with open(filename, encoding =\'utf-8\') as file:\n    sentences = []\n    for sentence in file.read().split("\n"):\n      #print(word)\n      if len(sentence.strip()) > 4:\n        sentences.append(torch.tensor(language.getSentence(sentence.strip())))\n  return sentences\n'

In [35]:
from torch.nn.utils.rnn import pad_sequence
import random

In [36]:
#sequences = generate_seq('data/shakespeare.txt')


In [37]:
class Shakespeare(Dataset):
  def __init__(self, dir, transform = None):
    self.sequences = self.generate_seq(dir)
    self.transform = transform
    self.PAD_token = 2
    self.EOS_token = 1
    self.SOS_token = 0

  def generate_seq(self, filename):
    with open(filename, encoding ='utf-8') as file:
      sentences = []
      for sentence in file.read().split("\n"):
        #print(word)
        if len(sentence.strip()) > 4:
          sentences.append(sentence.strip())
    return sentences
  
  def __len__(self):
    return len(self.sequences)
  
  def __getitem__(self, idx):
    #assert isinstance(idx, list)
    out = random.sample(self.sequences, idx)
    seq_out = []
    for i in range(len(out)):
      seq_out.append(torch.tensor(language.getSentence(out[i])))

    out = pad_sequence(seq_out, padding_value = PAD_token).permute(1,0)
    out = F.pad(out, (0, 1), value = EOS_token)
    out = F.pad(out, (1, 0), value = SOS_token)
    return out

In [38]:
dataset = Shakespeare(dir = 'data/shakespeare.txt')

In [39]:
def get_batch(dataset, bs):
  ds = len(dataset)
  assert bs <= ds, 'Batch size must be less than or equal to input dim'
  perm = torch.randperm(ds)
  for i in range(ds//bs):
    yield dataset[bs]

In [40]:
#Transformer
class Transformer(nn.Module):
    def __init__(self, heads, src_len, trg_len, d_model, vocab_size, n_blocks = 4):
        super(Transformer, self).__init__()

        self.heads = heads
        self.src_len = src_len
        self.trg_len = trg_len
        self.d_model = d_model
        self.n_blocks = n_blocks
        self.vocab_size = vocab_size 

        self.encoders = nn.ModuleList([Encoder(self.heads, self.src_len, self.d_model) for i in range(self.n_blocks)]) 
        self.decoders = nn.ModuleList([Decoder(self.heads, self.trg_len, self.d_model) for i in range(self.n_blocks)]) 

        self.pe = PositionalEncoding(self.src_len, self.d_model)
        self.embed = nn.Embedding(self.vocab_size, self.d_model)

        self.linear = nn.Linear(self.d_model, self.vocab_size)

    def forward(self, src_idx, trg_idx):
        bs = src_idx.size(0)
        src_out = self.pe()[:self.src_len,:] + self.embed(src_idx)

        for i in range(len(self.encoders)):
            src_out = self.encoders[i](src_out)

        trg_out = self.pe()[:self.trg_len,:] + self.embed(trg_idx)
        for i in range(len(self.decoders)):
            trg_out = self.decoders[i](trg_out, src_out)

        trg_out = self.linear(trg_out)
        #trg_out = F.softmax(trg_out, dim = -1)
        #trg_out = trg_out.view(bs, self.trg_len, self.vocab_size)
        return trg_out

#class GPT
class GPT(nn.Module):
    def __init__(self , heads, trg_len, d_model, vocab_size, n_blocks = 4):
        super(GPT, self).__init__()

        self.heads = heads
        self.trg_len = trg_len
        self.d_model = d_model
        self.n_blocks = n_blocks
        self.vocab_size = vocab_size 

        #self.encoders = nn.ModuleList([Encoder(self.heads, self.src_len, self.d_model) for i in range(self.n_blocks)]) 
        self.decoders = nn.ModuleList([Decoder(self.heads, self.trg_len, self.d_model) for i in range(self.n_blocks)]) 

        self.pe = PositionalEncoding(self.trg_len, self.d_model)
        self.embed = nn.Embedding(self.vocab_size, self.d_model)

        self.linear = nn.Linear(self.d_model, self.vocab_size)

    def forward(self, idx):
        bs = idx.size(0)

        trg_out = self.pe()[:idx.size(1),:] + self.embed(idx)

        for i in range(len(self.decoders)):
            trg_out = self.decoders[i](trg_out, trg_out)

        trg_out = self.linear(trg_out)
        #trg_out = F.softmax(trg_out, dim = -1)
        #trg_out = trg_out.view(bs, self.trg_len, self.vocab_size)
        return trg_out

#Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, seq_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        
        self.register_buffer('PE', self.generate_pe())

    def forward(self):
        return self.PE

    def generate_pe(self):
        pos_enc = torch.zeros((self.seq_len, self.d_model))
        den = torch.pow(10000, 2 * (torch.arange(pos_enc.size(1)//2)) / self.d_model)
        pos_enc[:, 0::2] = torch.sin(torch.arange(pos_enc.size(0)).unsqueeze(1)/den)
        pos_enc[:, 1::2] = torch.cos(torch.arange(pos_enc.size(0)).unsqueeze(1)/den)
        return pos_enc
#Encoder
class Encoder(nn.Module):
    def __init__(self, heads,seq_len, d_model):
        super(Encoder, self).__init__()
        self.heads = heads
        self.d_model = d_model
        self.multihead_attn = MultiHeadAttention(self.heads, self.d_model)
        self.ln1 = nn.LayerNorm(d_model)
        self.ffnn = FFNN(self.d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(0.8)
        self.dropout2 = nn.Dropout(0.8)

    def forward(self, inputs):
        inputs = self.ln1(self.dropout1(self.multihead_attn(inputs, inputs, inputs)) + inputs)
        inputs = self.ln2(self.dropout2(self.ffnn(inputs)) + inputs)
        return inputs

#Decoder
class Decoder(nn.Module):
    def __init__(self, heads, trg_len, d_model):
        super(Decoder, self).__init__()

        self.heads = heads
        self.d_model = d_model
        self.trg_len = trg_len 

        self.mask_attn = MultiHeadAttention(self.heads, self.d_model)
        self.ln1 = nn.LayerNorm(d_model)

        self.multihead_attn = MultiHeadAttention(self.heads, self.d_model)
        self.ln2 = nn.LayerNorm(d_model)

        self.ffnn = FFNN(self.d_model)
        self.ln3 = nn.LayerNorm(d_model)

        self.dropout1 = nn.Dropout(0.8)
        self.dropout2 = nn.Dropout(0.8)
        self.dropout3 = nn.Dropout(0.8)
        #self.generate_mask()

    def forward(self, inputs, keys):

        inputs = self.ln1(self.dropout1(self.multihead_attn(inputs, inputs, inputs, mask = self.generate_mask(inputs.size(1))) + inputs))
        inputs = self.ln2(self.dropout2(self.multihead_attn(inputs, keys, keys)) + inputs)
        inputs = self.ln3(self.dropout3(self.ffnn(inputs)) + inputs)
        return inputs

    def generate_mask(self, trg_len):
        mask = torch.ones((trg_len, trg_len))
        mask = torch.triu(mask, diagonal = 1) * -100000
        self.register_buffer('mask', mask)
        return mask
#FFNN
class FFNN(nn.Module):
    def __init__(self, d_model):
        super(FFNN, self).__init__()
        self.d_model = d_model
        self.ln1 = nn.Linear(self.d_model, self.d_model * 4)
        self.ln2 = nn.Linear(self.d_model * 4, self.d_model)
        self.lnorm = nn.LayerNorm(d_model)

    def forward(self, x):
        inputs = F.relu(self.ln1(x))
        inputs = self.lnorm(self.ln2(inputs) + x)
        return inputs

#Multihead Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model):
        super(MultiHeadAttention, self).__init__()

        self.heads = heads
        self.d_model = d_model

        assert self.d_model % self.heads == 0, 'heads has to be a multiple of d_model'
        self.query = nn.Linear(self.d_model, self.d_model, bias = False)
        self.key = nn.Linear(self.d_model, self.d_model, bias = False)
        self.value = nn.Linear(self.d_model, self.d_model, bias = False)

        self.ln = nn.Linear(self.d_model, self.d_model)

    def forward(self, query, key, value, mask = None):
        bs = query.size(0)
        seq_len = query.size(1)

        #print(query.shape)
        #inputs = inputs.view(-1, self.d_model)
        #print(self.query(query).shape)
        query = self.query(query).view(bs, seq_len, self.heads, self.d_model//self.heads)
        key = self.key(key).view(bs, seq_len, self.heads, self.d_model//self.heads)
        value = self.value(value).view(bs, seq_len, self.heads, self.d_model//self.heads)

        
        #print(query.shape)
        query = query.permute(0,2,1,3).contiguous().view(bs * self.heads, seq_len, self.d_model//self.heads)
        key = key.permute(0,2,1,3).contiguous().view(bs * self.heads, seq_len, self.d_model//self.heads)
        value = value.permute(0,2,1,3).contiguous().view(bs * self.heads, seq_len, self.d_model//self.heads)

        inputs = sccaled_dot_product_attention(query, key, value, mask) 
        inputs = inputs.view(bs, self.heads, seq_len, self.d_model//self.heads).permute(0,2,1,3).contiguous()
        inputs = inputs.view(bs, seq_len, self.heads * self.d_model//self.heads)
        inputs = self.ln(inputs)
        return inputs

#scaled dot product attention
def sccaled_dot_product_attention(query, key, value, mask):
    if mask is not None:
        attn = torch.bmm(query, key.transpose(-1,-2))/torch.sqrt(torch.FloatTensor([key.size(-1)]).to(key.device)) + mask.to(key.device).detach()
    else:
        attn = torch.bmm(query, key.transpose(-1,-2))/torch.sqrt(torch.FloatTensor([key.size(-1)]).to(key.device))

    attn = torch.softmax(attn, dim = -1)
    
    output = torch.bmm(attn, value)
    return output

In [50]:
from torch.distributions.categorical import Categorical
def generate_(model, seq_len, language):
  with torch.no_grad():
    model.eval()
    seq = [SOS_token]
    for i in range(seq_len):
      out = model(torch.tensor(seq).unsqueeze(1).to('cuda'))
      out = F.softmax(out, dim = -1)
      m = Categorical(out)
      token = m.sample().squeeze(1).tolist()[-1]
      #token = torch.argmax(out, dim = -1).squeeze(1).tolist()[-1]
      if token == 1:
        break
      if token not in [0,1,2]:
        seq.append(token)
    return language.getListIndex(seq[1:])

In [51]:
model = GPT(8, 256, 800, vocab_size, n_blocks = 6).to('cuda')
model = torch.load('model_weights.pth')
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_token)

In [ ]:
losses = []
batch_size = 15
for i in range(100):
    total_loss = 0.0
    den = 0.0
    for data in get_batch(dataset, batch_size):
        model.train()
        input_ = data[:, :-1].to('cuda')
        target_ = data[:, 1:].to('cuda')

        optimizer.zero_grad()

        out = model(input_)
        #print(out.view(-1, vocab_size))
        #print(target_.view(-1))
        loss = loss_fn(out.view(-1, vocab_size), target_.view(-1))

        
        loss.backward()

        optimizer.step()
        #print(loss.item())
        den += input_.size(1)

        total_loss += loss.item()
    if i % 10 == 0 or i == 0:
        print(total_loss / den)
        losses.append(total_loss / den)
        torch.save(model, 'model_weights.pth')
        print(' '.join(generate_(model, 100, language)))


  

In [47]:

torch.save(model, 'model_weights.pth')
print(' '.join(generate_(model, 10, language)))

is gain will . . yokefellow heart is
